In [6]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/ocean/projects/asc170022p/mtragoza/lung-project'

In [7]:
import sys, os
import numpy as np
import xarray as xr

In [8]:
%autoreload
import data
case = data.Emory4DCTCase(data_root='data/Emory-4DCT', case_name='Case1Pack', phases=[0, 50])
case.load_niftis()
case

Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz


Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', 2 phases)

In [9]:
def register_corrfield(case, fixed_phase, moving_phase):

    image_dir = case.nifti_dir.resolve()
    mask_dir = case.mask_dir.resolve()
    output_dir = case.case_dir.resolve() / 'CorrField'
    os.makedirs(output_dir, exist_ok=True)
    
    fixed_name = f'case{case.case_id}_T{fixed_phase:02d}'
    fixed_image = image_dir / (fixed_name + '.nii.gz')
    fixed_mask = mask_dir / fixed_name / 'lung_combined_mask.nii.gz'
    
    moving_name = f'case{case.case_id}_T{moving_phase:02d}'
    moving_image = image_dir / (moving_name + '.nii.gz')
    
    reg_name = f'case{case.case_id}_T{moving_phase:02d}_T{fixed_phase:02d}'
    output_path = output_dir / reg_name

    try:
        cwd = os.getcwd()
        os.chdir('../Lung250M-4B')
        import corrfield
        args = corrfield.parse_args(f'-F {fixed_image} -M {moving_image} -m {fixed_mask} -O {output_path}'.split())
        corrfield.main(args)
    except:
        os.chdir(cwd)
        raise
    finally:
        os.chdir(cwd)

In [12]:
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [11]:
register_corrfield(case, moving_phase=50, fixed_phase=0)

AttributeError: module 'corrfield' has no attribute 'parse_args'

In [25]:
import nibabel as nib
import xarray as xr
import hvplot.xarray

a = nib.load('/ocean/projects/asc170022p/mtragoza/lung-project/data/Emory-4DCT/Case1Pack/CorrField/case1_T00_T50.nii.gz')

xr.DataArray(
    a.get_fdata().astype(np.float32)[0],
    dims=['x', 'y', 'z', 'component'],
    coords={
        'x': np.arange(256) * 0.97,
        'y': np.arange(256) * 0.97,
        'z': np.arange(94) * 2.5,
        'component': ['x', 'y', 'z']
    }
).hvplot(groupby=['y', 'component'], x='x', y='z', data_aspect=1, clim=(-10, 10), cmap='seismic')

:DynamicMap   [y,component]
   :Image   [x,z]   (value)

In [ ]:
case.view(groupby=['phase', 'y'], x='x', y='z')

In [ ]:
%%time

spatial_shape = [case.array.sizes[d] for d in 'xyz']
spatial_coords = [case.array.coords[d] for d in 'xyz']

input_points = utils.nd_coords(spatial_coords)
output_points = F.apply(input_points)
U = output_points - input_points

displacement = xr.DataArray(
    U.reshape(spatial_shape + [3]),
    dims=['x', 'y', 'z', 'component'],
    coords={
        'x': case.array.x,
        'y': case.array.y,
        'z': case.array.z,
        'component': ['x', 'y', 'z']
    }
)
displacement

In [ ]:
data.view_array(displacement, groupby=['y', 'component'], x='x', y='z')

In [13]:
import torch
import torch.nn.functional as F

In [19]:
F.affine_grid(torch.eye(2, 3).unsqueeze(0), (1, 1, 3, 3), align_corners=True).shape

torch.Size([1, 3, 3, 2])

In [ ]:
img_mov_warped = F.grid_sample(
    img_mov,
    F.affine_grid(
        torch.eye(3, )
    )
)

In [ ]:
sys.path.append('../Lung250M-4B')
import corrfield

corrfield.utils.

In [15]:


F.grid_sample(
    input=torch.zeros(1,1,),
    grid=torch.zeros(3),
    mode='bilinear'
).shape

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/python3.10/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


RuntimeError: grid_sampler(): expected grid to have size -1 in last dimension, but got grid with sizes [3]